<center>
    <h1>Práctica 1 - Parte II. Análisis de datos tabulados: reparto de escaños al Congreso de los Diputados</h1>
    <h3>Programación - Grado en Ciencia de Datos</h3>
    <h3>Universitat Politècnica de València</h3>
</center>

**Práctica realizada por:**

<a id='indice'></a>
## Índice
1. ### [Objetivos](#objetivos)
1. ### [Legislación electoral española: circunscripciones y ley D'Hont](#legislacion_electoral)
1. ### [Funciones para reparto de escaños según la ley D'Hont](#funciones_dhont)
1. ### [Función para mostrar gráficas de resultados electorales](#funcion_resultados)
1. ### [Actividad 1: Cargar datos en un dataframe](#act1)
1. ### [Actividad 2: Porcentaje de participación por circunscripción](#act2)
1. ### [Actividad 3: Número de escaños por circunscripción ](#act3)
1. ### [Actividad 4: Número de escaños por comunidad](#act4)
1. ### [Actividad 5: Resultados por circunscripción](#act5)
1. ### [Actividad 6: Resultados para el congreso de los diputados](#act6)
1. ### [Actividad 7: Coste en votos de cada escaño](#act7)

<a id='objetivos'></a>
## Objetivos:
- Aprender a analizar y visualizar datos tabulados (estructurados en filas y columnas) mediante la realización de operaciones básicas (ordenación, selección, agrupamiento, agregación, ...)
- Aprender a usar los tipos de datos **DataFrame** y **Series** de la biblioteca pandas.
- Usar la librería **matplotlib.pyplot** para representar gráficamente los datos del estudio.

<a id='legislacion_electoral'></a>
## Legislación electoral española: división en circunscripciones y asignación de diputados mediante la Ley D'Hont

El congreso se compone de 350 diputados, que representan a 52 **circunscripciones** (las 50 provincias de España más las ciudades autónomas de Ceuta y Melilla). Cada circunscripción aporta un número de diputados. Concretamente, Ceuta y Melilla aportan un diputado cada una, y el resto de provincias lo hace en proporción a su población, con un mínimo de dos diputados. Por ejemplo, Madrid aporta 31 diputados, Valencia 16 o Soria 2.

En España se utiliza la **ley D'Hont** para asignar los diputados (o escaños) que le corresponden a cada partido en función de los votos obtenidos. Para ello, en primer lugar se excluye de cada circunscripción a las candidaturas que no hayan obtenido al menos un 3% de los votos válidos (votos a candidaturas + votos en blanco). A continuación, el reparto se hace siguiendo el siguiente algoritmo:

Se crea una tabla con tantas filas como partidos con opción a escaño y tantas columnas como escaños a repartir. En la primera columna se almacena el número de votos obtenido por cada partido, en la segunda el número de votos dividido entre dos, en la tercera el número de votos entre tres, y así sucesivamente. A continuación se obtienen los n valores máximos de esta tabla, siendo n el número de escaños a repartir. Por cada uno de estos máximos, se asigna un escaño al partido correspondiente. En la siguiente figura se muestra un ejemplo, en el que el partido A obtendría 2 diputados, el B otros 2 y el C 1.

![Figura reparto escaños](reparto_escanos.gif)

[Volver al índice](#indice)

<a id='funciones_dhont'></a>
## Funciones para realizar el reparto de escaños según la ley D'Hont

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def reparto_escaños(votos_validos, tot_escaños, votos_partidos, porcentaje_min=3):
    """
    Hace el reparto de escaños aplicando la Ley D'Hont.
    Parámetros de entrada:
       - votos_validos: número de votos válidos (votos a candidatura + votos en blanco)
       - tot_escaños: número de escaños a repartir
       - votos_partidos: diccionario del tipo {'Nombre partido':votos obtenidos}
       - porcentaje_min: porcentaje mínimo de votos para que una candidatura tenga opción a escaño
    Devuelve:
       - diccionario del tipo {'Nombre partido':diputados asignados}
    """
    # En primer lugar determinar las candidaturas que han superado un procentaje mínimo de votos:
    candidatos = []
    for partido,votos in votos_partidos.items():
        if votos * 100 / votos_validos >= porcentaje_min:
            candidatos.append(partido)

    # Creamos una lista auxiliar en la que pondremos tuplas (votos/i, partido) con i: 1..tot_escaños
    aux = []
    for partido in candidatos:
        for i in range(tot_escaños):
            aux.append((votos_partidos[partido]/(i+1),partido))

    # Ordenamos la lista aux de mayor a menor:
    aux.sort(reverse=True)

    # Tomamos las 'tot_escaños' primeras tuplas de aux y creamos el diccionario 'resultado'
    resultado = {}
    for score,partido in aux[:tot_escaños]:
        if partido in resultado:
            resultado[partido] += 1
        else:
            resultado[partido] = 1

    return resultado                 

<a id='funcion_resultados'></a>
## Función para mostrar los diputados obtenidos por cada candidatura

In [ ]:
import matplotlib.pyplot as plt

def get_color(partido):
    dic_colores = {'PSOE':'red', 'PP':'blue', "Cs":'darkorange', 'UP':'darkmagenta', 'ERC':'orange', 'VOX':'limegreen', 'PNV':'green'}
    for clave in dic_colores:
        if partido in clave or clave in partido:
            return dic_colores[clave]
    return 'gray'
    
def plot_escaños(dic_partidos_escaños): 
    """
    Muestra los diputados obtenidos por cada candidatura.
    Parámetros de entrada:
       - dic_partidos_escaños: diccionario del tipo {'Nombre partido':diputados asignados} 
                               (esto es, valor devuelto por la función reparto_escanyos)
    """
    # Guardar en tres listas separadas los nombres de partido, los diputados asignados
    # y los colores corporativos, todo ello ordenado por número de diputados
    partidos = sorted(dic_partidos_escaños, key=dic_partidos_escaños.get, reverse=True)
    escaños = sorted(dic_partidos_escaños.values(), reverse=True)
    colores = []
    for partido in partidos:
        colores.append(get_color(partido))
    
    plt.figure(figsize=(12, 3))
    
    # MOSTRAR GRAFICA DE TARTA (subfigura 1)
    
    # Eliminar labels de partidos con poca representación ya que no caben todas en el gráfico de tarta
    labels = partidos[:]
    tot_escaños = sum(escaños)
    for i in range(len(labels)):
        if escaños[i]/tot_escaños < 0.02: # Borrar etiqueta de partidos con menos de un 2% de escaños
            labels[i] = ''
            
    plt.subplot(1, 2, 1)           
    plt.pie(escaños, labels=labels, colors=colores, autopct=lambda val: int(np.round(val/100.*sum(escaños), 0)) if val>5 else "")
        
    # Dibujar circulo en el centro para hacer un donut
    centre_circle = plt.Circle((0,0),0.45,color='black', fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    # Establecer aspect ratio 'equal' para que tenga forma de círculo y no de ovalo
    plt.axis('equal')
    
    # MOSTRAR GRAFICA DE BARRAS (subfigura 2)
    plt.subplot(1, 2, 2)
    for i, v in enumerate(escaños):
        plt.text(i-0.1, v + 0.1, str(v))
        
    plt.bar(partidos,escaños, alpha=1, color=colores)
    plt.xticks(rotation=25, ha='right')
    plt.yticks(range(1, max(escaños)+2))
    plt.gca().set_yticklabels([])
    
    plt.subplots_adjust( wspace=1)
    plt.show(); 


<a id='act1'></a>
## ACTIVIDAD 1: Cargar datos en un DataFrame

Almacena el fichero "resultados_elecciones_2019.csv" en un dataframe de pandas y muestra las 5 primeras filas. El fichero utiliza como separador de campos el punto y coma, por lo que tendrás que especificar la opción sep=';' para leer correctamente el fichero.

*Fuente del archivo de datos: [http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html?method=inicio](http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html?method=inicio)*

[Volver al índice](#indice)

In [ ]:
import pandas as pd

df = pd.read_csv('resultados_elecciones_2019.csv', delimiter=';')

df.head()

<a id='act2'></a>
## ACTIVIDAD 2: Porcentaje de participación por circunscripción (provincia)

**Conceptos a trabajar:**
- Añadir nuevas columnas a un dataframe
- Ordenar un dataframe
- Mostrar gráfico de barras con las etiquetas de una columna y los datos de otra

Consulta cómo hacer estas operaciones en el tutorial `pandas_dataframe.ipynb` disponible en PoliformaT sección Tutoriales.

**Tareas a realizar:**
1. A partir de las columnas 'Total censo electoral' y 'Total votantes', crea una nueva columna en el dataframe para almacenar el porcentaje de participación en cada circunscripción.
1. Ordena las filas del dataframe según el valor de esta nueva columna
1. Crea un gráfico de barras horizontal con la función plt.barh, en la que el primer parámetro deberá ser los valores de la columna 'Nombre de Provincia' y el segundo los valores de la nueva columna con los porcentajes de participación.
1. Personaliza la figura:
    - Usa la instrucción plt.figure(figsize=(10,25)) para adecuar el tamaño
    - Usa la instrucción plt.grid(axis='x') para mostrar las líneas verticales de una rejilla
    - Usa la instrucción plt.title para poner un título a la gráfica
    
[Volver al índice](#indice)    

In [ ]:
# Crea nueva columna
df['Porcentaje participacion'] = 100 * df['Total votantes'] / df['Total censo electoral']

# Ordena el dataframe por Porcentaje participacion
df = df.sort_values(by='Porcentaje participacion', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

elec = df.groupby("Nombre de Provincia")["Porcentaje participacion"].mean().sort_values()


# Graficar ds con plt.barh.
# El orden de las instrucciones importa. La primera es el tamano siempre
plt.figure(figsize=(10,25))
plt.title("Porcentaje participacion por provincias")
plt.barh(elec.index, elec.values)
plt.grid(axis='x')

<a id='act3'></a>
## ACTIVIDAD 3: Número de escaños por circunscripción (provincia)

**Conceptos a trabajar:**
- Reforzar los mismos conceptos trabajados en la actividad 2

**Tareas a realizar:**

- Muestra una gráfica similar a la de la actividad 2, en este caso con el número de escaños a repartir en cada circunscripción. Ordena los datos en función de dicho número de escaños.

[Volver al índice](#indice)

In [ ]:
import matplotlib.pyplot as plt

elec = df.groupby("Nombre de Provincia")["Escaños"].mean().sort_values()

# Graficar ds con plt.barh
# El orden de las instrucciones importa. La primera es el tamano siempre
plt.figure(figsize=(10,25))
plt.title("Escaños por provincia")
plt.barh(elec.index, elec.values)
plt.grid(axis='x')

<a id='act4'></a>
## ACTIVIDAD 4: Número de escaños por comunidad

**Conceptos a trabajar:**
- Agrupar datos

Consulta cómo hacer estas operaciones en el tutorial `pandas_dataframe.ipynb` mencionado anteriormente.

**Tareas a realizar:**
- Muestra una gráfica similar a la de la actividad 3, pero mostrando en este caso el número total de escaños por Comunidad Autónoma. Para ello debrás utilizar la función *groupby* para agrupar filas en función de la Comunidad, junto con la función *sum* para obtener la suma de escaños en cada comunidad.

[Volver al índice](#indice)

In [ ]:
import matplotlib.pyplot as plt

df_agrupado = df.groupby("Nombre de Comunidad").sum()

elec = df.groupby("Nombre de Comunidad")["Escaños"].sum().sort_values()

# Graficar ds con plt.barh
# El orden de las instrucciones importa. La primera es el tamano siempre
plt.figure(figsize=(10,25))
plt.title("Escaños por comunidad")
plt.barh(elec.index, elec.values)
plt.grid(axis='x')

<a id='act5'></a>
## ACTIVIDAD 5: Resultados por circunscripción

**Conceptos a trabajar:**
- Iterar sobre las filas de un dataframe

**Tareas a realizar:**

Muestra, para cada provincia (circunscripción) una gráfica con el reparto de escaños en dicha provincia. Observa que cada fila del dataframe corresponde a una provincia distinta, por lo que será necesario iterar sobre cada fila para realizar esta operación. Esto puede hacerse con el bucle:

`for index, row in df.iterrows():`

donde `row` será un objeto de tipo `Series` con el contenido de cada fila.

Para cada fila:
1. Obtén el nombre de la provincia, los votos válidos y el número de escaños
1. Crea un diccionario donde las claves sean los nombres de los partidos y los valores los votos obtenidos
1. Obtén la asignación de escaños invocando adecuadamente al método reparto_escaños
1. Muestra un texto con el nombre de la provincia y el total de escaños a repartir y a continuación muestra la gráfica con los resultados invocando adecuadamente al método plot_escaños

[Comprueba que tus resultados coinciden con los publicados en este enlace](http://resultados-elecciones.rtve.es/generales/2019/congreso/)

[Volver al índice](#indice)

In [ ]:
print("NOTA: Los resultados salen ordenados por orden de participacion de mayor a menor", end='\n\n')

for index, row in df.iterrows():
    nombre_provincia = row[2]
    votos_validos = row[8]
    num_escanos = row[3]

    # Crear un diccionario donde las claves son los nombres de partidos y los valores los votos obtenidos. Solo para la provincia actual
    votos_partidos = {}
    i = 12
    for votos in row[12:]:
        if votos > 0: # Si el partido tiene votos en esa provincia
            # Conseguir el nombre del partido
            partido = df.columns[i]
            votos_partidos[partido] = votos
        i += 1
    
    # Obten la asignacion de escanos
    escanos = reparto_escaños(votos_validos, num_escanos, votos_partidos)

    # Muestra un texto con el nombre de la provincia y el total de escaños a repartir
    # A continuación muestra la gráfica con los resultados invocando adecuadamente al método plot_escaños
    print(f"Provincia: {nombre_provincia}   Escaños: {num_escanos}")
    plot_escaños(escanos)
    


    


<a id='act6'></a>
## ACTIVIDAD 6: Resultados para el congreso de los diputados

Muestra una gráfica con el total de diputados obtenidos por cada partido al congreso de los diputados. Para ello deberás ir calculando los diputados que ha obtenido cada partido en cada circunscripción, tal y como has hecho en la actividad anterior, e ir acumulando (sumando) dicho número de diputados en otro diccionario. Finalmente, ejecuta la función plot_escaños pasándole el diccionario obtenido.

[Volver al índice](#indice)

In [ ]:
# Diccionario de escanos totales
escanos_congreso = {}

# Calcula los escanos en cada provincia
for index, row in df.iterrows():
    nombre_provincia = row[2]
    votos_validos = row[8]
    num_escanos = row[3]

    # Crear un diccionario donde las claves son los nombres de partidos y los valores los votos obtenidos. Solo para la provincia actual
    votos_partidos = {}
    i = 12
    for votos in row[12:]:
        if votos > 0: # Si el partido tiene votos en esa provincia
            # Conseguir el nombre del partido
            partido = df.columns[i]
            votos_partidos[partido] = votos
        i += 1
    
    # Obten la asignacion de escanos
    dicc_escanos_provincia = reparto_escaños(votos_validos, num_escanos, votos_partidos)

    # Asigna los escanos de la provincia al congreso
    for partido, escanos_provincia in dicc_escanos_provincia.items():
        # Coger la suma de diputados para ese partido ya existente
        suma_diputados = escanos_congreso.get(partido)
        # Si el partido ya tenia escanos:
        if suma_diputados:
            escanos_congreso[partido] = suma_diputados + escanos_provincia
        else: # Si el partido no tenia escanos
            escanos_congreso[partido] = escanos_provincia



# Grafica los escanos totales
plot_escaños(escanos_congreso)

<a id='act7'></a>
## ACTIVIDAD 7: Coste en votos de cada escaño

**Conceptos a trabajar:**
- Sumar columnas


**Tareas a realizar:**

En esta actividad vamos a mostrar cuántos votos le cuesta a cada partido obtener un escaño (votos obtenidos / escaños conseguidos). Para ello, realiza las siguientes tareas:


1. A partir del diccionario con los resultados de la actividad anterior, crea otro diccionario que contenga, para cada partido, el total de votos obtenidos entre el número de escaños conseguidos.

1. Muestra una gráfica de barras horizontal con los items de este nuevo diccionario. Dado que la función `plt.barh` trabaja con listas y no con diccionarios, deberás almacenar los valores del diccionario en listas separadas. Para ello puedes utilizar `list(d.keys())` y `list(d.values())` para obtener, respectivamente, las etiquetas y los valores, siendo `d` es el diccionario con los resultados.

1. Añade a la gráfica un grid con líneas verticales y un título, tal y como has hecho en actividades anteriores.

[Volver al índice](#indice)

In [ ]:
# Crear otro diccionario que contiene para cada partido el coste de sus escanos: total_votos / num_escanos
coste_escanos = {}
for partido, escanos in escanos_congreso.items():
    # Conseguir el total de votos
    ds = df.loc[:, partido]
    total_votos = ds.sum()
    # Anadir a coste_escanos el coste de los escanos
    coste_escanos[partido] = total_votos / escanos

# Separar en listas
ds = pd.Series(coste_escanos).sort_values(ascending=True)
#partidos = list(coste_escanos.keys())
#coste = list(coste_escanos.values())

# Grafica con plt.barh
plt.figure(figsize=(10,10))
plt.title("Coste de escaños por partido")
plt.barh(ds.index, ds.values)
plt.grid(axis='x')

